In [1]:
import pandas as pd

# Define the file path
file_path = r'C:\Users\jbuck\Player-Recommendation-System\Player Metrics\Team Stats\team_stats_for_metric.csv'

# Load the CSV file into a DataFrame
teams_df = pd.read_csv(file_path)

import pandas as pd

# Define the file path
file_path = r'C:\Users\jbuck\Player-Recommendation-System\Player Metrics\Team Stats\opp_team_stats_for_metric.csv'

# Load the CSV file into a DataFrame
opp_teams_df = pd.read_csv(file_path)

In [2]:


# Rename the columns in opp_team_stats_df, add '_allowed' suffix
opp_teams_df.rename(columns=lambda x: x + '_allowed' if x not in ['team', 'league'] else x, inplace=True)

# Merge DataFrames on 'team' and 'league'
merged_df = pd.merge(teams_df, opp_teams_df, on=['team', 'league'])
# Correct columns after an error has been made
merged_df.columns = [col.replace('_allowed_allowed', '_allowed') if '_allowed_allowed' in col else col for col in merged_df.columns]

# Display the merged DataFrame
print(merged_df.head())

               league  season         team  players_used  90s  Standard_Gls  \
0  ENG-Premier League    2324      Arsenal            25   27            65   
1  ENG-Premier League    2324  Aston Villa            28   27            56   
2  ENG-Premier League    2324  Bournemouth            27   26            35   
3  ENG-Premier League    2324    Brentford            28   27            38   
4  ENG-Premier League    2324     Brighton            27   27            46   

   Standard_Sh  Standard_SoT  Standard_SoT%  Standard_Sh/90  ...  \
0          453           148           32.7           16.78  ...   
1          394           146           37.1           14.59  ...   
2          343           112           32.7           13.19  ...   
3          340           111           32.6           12.59  ...   
4          404           157           38.9           14.96  ...   

   Pass Types_TI_allowed  Pass Types_CK_allowed  Corner Kicks_In_allowed  \
0                    498                

In [3]:
import pandas as pd

# Assuming df is your DataFrame

# Print each column name on a new line
for column in merged_df.columns:
    print(column)


league
season
team
players_used
90s
Standard_Gls
Standard_Sh
Standard_SoT
Standard_SoT%
Standard_Sh/90
Standard_SoT/90
Standard_G/Sh
Standard_G/SoT
Standard_Dist
Standard_FK
Standard_PK
Standard_PKatt
Expected_xG
Expected_npxG
Expected_npxG/Sh
Expected_G-xG
Expected_np:G-xG
url
League
season.1
players_used.1
90s.1
SCA_SCA
SCA_SCA90
SCA Types_PassLive
SCA Types_PassDead
SCA Types_TO
SCA Types_Sh
SCA Types_Fld
SCA Types_Def
GCA_GCA
GCA_GCA90
GCA Types_PassLive
GCA Types_PassDead
GCA Types_TO
GCA Types_Sh
GCA Types_Fld
GCA Types_Def
url.1
League.1
season.2
players_used.2
90s.2
Tackles_Tkl
Tackles_TklW
Tackles_Def 3rd
Tackles_Mid 3rd
Tackles_Att 3rd
Challenges_Tkl
Challenges_Att
Challenges_Tkl%
Challenges_Lost
Blocks_Blocks
Blocks_Sh
Blocks_Pass
Int
Tkl+Int
Clr
Err
url.2
League.2
season.3
players_used.3
90s.3
Performance_CrdY
Performance_CrdR
Performance_2CrdY
Performance_Fls
Performance_Fld
Performance_Off
Performance_Crs
Performance_Int
Performance_TklW
Performance_PKwon
Performance_PKco

In [4]:
# Drop duplicate rows
merged_df = merged_df.drop_duplicates()

# Display the DataFrame after dropping duplicates
print("\nDataFrame after dropping duplicates:")
print(merged_df)


DataFrame after dropping duplicates:
                 league  season             team  players_used  90s  \
0    ENG-Premier League    2324          Arsenal            25   27   
1    ENG-Premier League    2324      Aston Villa            28   27   
2    ENG-Premier League    2324      Bournemouth            27   26   
3    ENG-Premier League    2324        Brentford            28   27   
4    ENG-Premier League    2324         Brighton            27   27   
..                  ...     ...              ...           ...  ...   
215          ARG-League    2324        Sarmiento            32   27   
216          ARG-League    2324         Talleres            31   27   
217          ARG-League    2324            Tigre            27   27   
218          ARG-League    2324          Tucumán            26   27   
219          ARG-League    2324  Vélez Sarsfield            27   27   

     Standard_Gls  Standard_Sh  Standard_SoT  Standard_SoT%  Standard_Sh/90  \
0              65          453

In [5]:
import pandas as pd



# Calculating the stats
# 1. Attacking third tackles per opposition defensive third touches
merged_df['att_tackles_per_def_touches'] = merged_df['Tackles_Att 3rd'] / merged_df['Touches_Def 3rd_allowed']

# 2. Inverse of opposition pass completion rate
merged_df['opp_pass_completion'] = merged_df['Total_Cmp%_allowed']

# 3. Opposition defensive third touches per middle third touch
merged_df['def_touches_per_mid_touches'] = merged_df['Touches_Def 3rd_allowed'] / merged_df['Touches_Mid 3rd_allowed']

# Display the new columns
print(merged_df)


                 league  season             team  players_used  90s  \
0    ENG-Premier League    2324          Arsenal            25   27   
1    ENG-Premier League    2324      Aston Villa            28   27   
2    ENG-Premier League    2324      Bournemouth            27   26   
3    ENG-Premier League    2324        Brentford            28   27   
4    ENG-Premier League    2324         Brighton            27   27   
..                  ...     ...              ...           ...  ...   
215          ARG-League    2324        Sarmiento            32   27   
216          ARG-League    2324         Talleres            31   27   
217          ARG-League    2324            Tigre            27   27   
218          ARG-League    2324          Tucumán            26   27   
219          ARG-League    2324  Vélez Sarsfield            27   27   

     Standard_Gls  Standard_Sh  Standard_SoT  Standard_SoT%  Standard_Sh/90  \
0              65          453           148           32.7         

In [6]:
import pandas as pd
import numpy as np

def cap_extreme_values(data, columns):
    for col in columns:
        upper_limit = data[col].quantile(0.99)
        lower_limit = data[col].quantile(0.01)
        data[col] = np.clip(data[col], lower_limit, upper_limit)
    return data

def log_transform(data, columns):
    for col in columns:
        offset = 1e-5  # Avoid log(0) issue
        data[col + '_log'] = np.log(data[col] + offset)
    return data

def calculate_z_scores(data, columns):
    z_scores_df = data.copy()
    for col in columns:
        if 'opp_pass_completion' in col:
            z_scores_df[col + '_z'] = -(z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()  # Inversion for pass completion
        else:
            z_scores_df[col + '_z'] = (z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()
    return z_scores_df

def compute_high_press_score(data, columns, weights):
    z_score_columns = [col + '_z' for col in columns]
    data['high_press_score'] = data[z_score_columns].dot(weights)
    min_score = data['high_press_score'].min()
    max_score = data['high_press_score'].max()
    data['high_press_score'] = ((data['high_press_score'] - min_score) / (max_score - min_score)) * 100
    return data

# Columns and weights configuration
high_press_columns = [
    'def_touches_per_mid_touches', 'opp_pass_completion',
    'att_tackles_per_def_touches'
]
weights_high_press = np.array([0.3, 0.3, 0.4])  # Weighting considering importance

# Load data and apply transformations
merged_df = cap_extreme_values(merged_df, high_press_columns)
merged_df = log_transform(merged_df, high_press_columns)
scores_df_high_press = calculate_z_scores(merged_df, high_press_columns)
scores_df_high_press = compute_high_press_score(scores_df_high_press, high_press_columns, weights_high_press)

# Merge into a new DataFrame
team_scores_df = merged_df[['team', 'league']].copy()  # Ensuring team and league are included
team_scores_df = team_scores_df.merge(scores_df_high_press[['team', 'league', 'high_press_score']],
                                      on=['team', 'league'], how='left')

# Display the DataFrame to check the results
print(team_scores_df.head())


          team              league  high_press_score
0      Arsenal  ENG-Premier League         77.614792
1  Aston Villa  ENG-Premier League         34.251561
2  Bournemouth  ENG-Premier League         53.723397
3    Brentford  ENG-Premier League         32.431239
4     Brighton  ENG-Premier League         65.279580


In [7]:
def view_top_players(df, score_column, num_players=10):
    """ Display the top players based on the given score column. """
    top_players = df.sort_values(by=score_column, ascending=False).head(num_players)
    return top_players

# Example usage to view the top 10 players for the 'Progressor' role
top_wcs = view_top_players(team_scores_df, 'high_press_score')
print(top_wcs)


               team              league  high_press_score
192  Argentinos Jun          ARG-League        100.000000
122   PSV Eindhoven      NED-Eredivisie         99.222159
42    Cercle Brugge      BEL-Pro League         98.879333
17        Tottenham  ENG-Premier League         91.080811
116       Feyenoord      NED-Eredivisie         89.648705
159      Bragantino          BRA-League         87.473639
148     Southampton    ENG-Championship         83.637795
187           Porto          POR-League         83.108475
212     River Plate          ARG-League         80.729252
175         Benfica          POR-League         80.579234


In [8]:
# Display descriptive statistics for the three score columns
descriptive_statistics = team_scores_df[['high_press_score']].describe()
print(descriptive_statistics)


       high_press_score
count        220.000000
mean          40.487914
std           19.868512
min            0.000000
25%           26.760787
50%           35.723383
75%           52.605384
max          100.000000


In [9]:
merged_df['opp_att_3rd_touches_per_opp_att_pen_touches'] = merged_df['Touches_Att 3rd_allowed'] / merged_df['Touches_Att Pen_allowed']
print(merged_df)


                 league  season             team  players_used  90s  \
0    ENG-Premier League    2324          Arsenal            25   27   
1    ENG-Premier League    2324      Aston Villa            28   27   
2    ENG-Premier League    2324      Bournemouth            27   26   
3    ENG-Premier League    2324        Brentford            28   27   
4    ENG-Premier League    2324         Brighton            27   27   
..                  ...     ...              ...           ...  ...   
215          ARG-League    2324        Sarmiento            32   27   
216          ARG-League    2324         Talleres            31   27   
217          ARG-League    2324            Tigre            27   27   
218          ARG-League    2324          Tucumán            26   27   
219          ARG-League    2324  Vélez Sarsfield            27   27   

     Standard_Gls  Standard_Sh  Standard_SoT  Standard_SoT%  Standard_Sh/90  \
0              65          453           148           32.7         

In [10]:
import pandas as pd
import numpy as np

def cap_extreme_values(data, columns):
    for col in columns:
        upper_limit = data[col].quantile(0.99)
        lower_limit = data[col].quantile(0.01)
        data[col] = np.clip(data[col], lower_limit, upper_limit)
    return data

def log_transform(data, columns):
    for col in columns:
        offset = 1e-5  # Avoid log(0) issue
        data[col + '_log'] = np.log(data[col] + offset)
    return data

def calculate_z_scores(data, columns):
    z_scores_df = data.copy()
    for col in columns:
        if 'Expected_npxG/Sh_allowed' in col:
            z_scores_df[col + '_z'] = -(z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()  # Inversion for pass completion
        else:
            z_scores_df[col + '_z'] = (z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()
    return z_scores_df

def compute_high_press_score(data, columns, weights):
    z_score_columns = [col + '_z' for col in columns]
    data['deep_block_score'] = data[z_score_columns].dot(weights)
    min_score = data['deep_block_score'].min()
    max_score = data['deep_block_score'].max()
    data['deep_block_score'] = ((data['deep_block_score'] - min_score) / (max_score - min_score)) * 100
    return data

# Columns and weights configuration
deep_block_columns = [
    'opp_att_3rd_touches_per_opp_att_pen_touches', 'Expected_npxG/Sh_allowed'
]
weights_deep_block = np.array([0.65, 0.35])  # Weighting considering importance

# Load data and apply transformations
merged_df = cap_extreme_values(merged_df, deep_block_columns)
merged_df = log_transform(merged_df, deep_block_columns)
scores_df_deep_block = calculate_z_scores(merged_df, deep_block_columns)
scores_df_deep_block = compute_high_press_score(scores_df_deep_block, deep_block_columns, weights_deep_block)

# Merge into a new DataFrame
team_scores_df = team_scores_df.merge(scores_df_deep_block[['team', 'league', 'deep_block_score']],
                                      on=['team', 'league'], how='left')

# Display the DataFrame to check the results
print(team_scores_df.head())


          team              league  high_press_score  deep_block_score
0      Arsenal  ENG-Premier League         77.614792         51.250689
1  Aston Villa  ENG-Premier League         34.251561          0.000000
2  Bournemouth  ENG-Premier League         53.723397         25.881742
3    Brentford  ENG-Premier League         32.431239         75.947849
4     Brighton  ENG-Premier League         65.279580          9.840267


In [11]:
def view_top_players(df, score_column, num_players=10):
    """ Display the top players based on the given score column. """
    top_players = df.sort_values(by=score_column, ascending=True).head(num_players)
    return top_players

# Example usage to view the top 10 players for the 'Progressor' role
top_wcs = view_top_players(team_scores_df, 'deep_block_score')
print(top_wcs)


             team              league  high_press_score  deep_block_score
1     Aston Villa  ENG-Premier League         34.251561          0.000000
127       Vitesse      NED-Eredivisie         40.348612          3.677082
113          Ajax      NED-Eredivisie         61.757722          7.412333
85           Köln      GER-Bundesliga         12.230084          9.139870
128      Volendam      NED-Eredivisie         19.963222          9.183729
79   Darmstadt 98      GER-Bundesliga         23.213660          9.542051
31     Las Palmas         ESP-La Liga         57.038022          9.672959
5         Burnley  ENG-Premier League         38.755368          9.740452
78         Bochum      GER-Bundesliga         54.441630          9.740452
4        Brighton  ENG-Premier League         65.279580          9.840267


In [12]:
merged_df['opp_switches_per_pass_att'] = merged_df['Pass Types_Sw_allowed'] / merged_df['Total_Att_allowed']
merged_df['opp_crosses_per_att_3rd_touch'] = merged_df['Pass Types_Crs_allowed'] / merged_df['Touches_Att 3rd_allowed']

print(merged_df)


                 league  season             team  players_used  90s  \
0    ENG-Premier League    2324          Arsenal            25   27   
1    ENG-Premier League    2324      Aston Villa            28   27   
2    ENG-Premier League    2324      Bournemouth            27   26   
3    ENG-Premier League    2324        Brentford            28   27   
4    ENG-Premier League    2324         Brighton            27   27   
..                  ...     ...              ...           ...  ...   
215          ARG-League    2324        Sarmiento            32   27   
216          ARG-League    2324         Talleres            31   27   
217          ARG-League    2324            Tigre            27   27   
218          ARG-League    2324          Tucumán            26   27   
219          ARG-League    2324  Vélez Sarsfield            27   27   

     Standard_Gls  Standard_Sh  Standard_SoT  Standard_SoT%  Standard_Sh/90  \
0              65          453           148           32.7         

In [13]:
import pandas as pd
import numpy as np

def cap_extreme_values(data, columns):
    for col in columns:
        upper_limit = data[col].quantile(0.99)
        lower_limit = data[col].quantile(0.01)
        data[col] = np.clip(data[col], lower_limit, upper_limit)
    return data

def log_transform(data, columns):
    for col in columns:
        offset = 1e-5  # Avoid log(0) issue
        data[col + '_log'] = np.log(data[col] + offset)
    return data

def calculate_z_scores(data, columns):
    z_scores_df = data.copy()
    for col in columns:
        z_scores_df[col + '_z'] = (z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()
    return z_scores_df

def compute_high_press_score(data, columns, weights):
    z_score_columns = [col + '_z' for col in columns]
    data['narrow_defense_score'] = data[z_score_columns].dot(weights)
    min_score = data['narrow_defense_score'].min()
    max_score = data['narrow_defense_score'].max()
    data['narrow_defense_score'] = ((data['narrow_defense_score'] - min_score) / (max_score - min_score)) * 100
    return data

# Columns and weights configuration
narrow_defense_columns = [
    'opp_crosses_per_att_3rd_touch', 'opp_switches_per_pass_att'
]
weights_narrow_defense = np.array([0.3, 0.7])  # Weighting considering importance

# Load data and apply transformations
merged_df = cap_extreme_values(merged_df, narrow_defense_columns)
merged_df = log_transform(merged_df, narrow_defense_columns)
scores_df_narrow_defense = calculate_z_scores(merged_df, narrow_defense_columns)
scores_df_narrow_defense = compute_high_press_score(scores_df_narrow_defense, narrow_defense_columns, weights_narrow_defense)

# Merge into a new DataFrame
team_scores_df = team_scores_df.merge(scores_df_narrow_defense[['team', 'league', 'narrow_defense_score']],
                                      on=['team', 'league'], how='left')

# Display the DataFrame to check the results
print(team_scores_df.head())


          team              league  high_press_score  deep_block_score  \
0      Arsenal  ENG-Premier League         77.614792         51.250689   
1  Aston Villa  ENG-Premier League         34.251561          0.000000   
2  Bournemouth  ENG-Premier League         53.723397         25.881742   
3    Brentford  ENG-Premier League         32.431239         75.947849   
4     Brighton  ENG-Premier League         65.279580          9.840267   

   narrow_defense_score  
0             47.643715  
1             48.477344  
2             16.406429  
3             18.068076  
4             17.682018  


In [14]:
def view_top_players(df, score_column, num_players=10):
    """ Display the top players based on the given score column. """
    top_players = df.sort_values(by=score_column, ascending=False).head(num_players)
    return top_players

# Example usage to view the top 10 players for the 'Progressor' role
top_wcs = view_top_players(team_scores_df, 'narrow_defense_score')
print(top_wcs)


              team              league  high_press_score  deep_block_score  \
17       Tottenham  ENG-Premier League         91.080811         17.004437   
207      Instituto          ARG-League         51.533278         63.548988   
217          Tigre          ARG-League         79.286600         47.869461   
42   Cercle Brugge      BEL-Pro League         98.879333         12.462457   
169  Internacional          BRA-League         39.369849         67.685414   
196       Belgrano          ARG-League         34.962998         52.792657   
39      Villarreal         ESP-La Liga         32.340554         45.500915   
35     Real Madrid         ESP-La Liga         42.825005         77.906301   
208          Lanús          ARG-League         55.103221         85.515126   
212    River Plate          ARG-League         80.729252         65.811461   

     narrow_defense_score  
17             100.000000  
207             98.525669  
217             90.173901  
42              86.336182  
1

In [15]:
# Display descriptive statistics for the three score columns
descriptive_statistics = team_scores_df[['narrow_defense_score', 'high_press_score', 'deep_block_score']].describe()
print(descriptive_statistics)


       narrow_defense_score  high_press_score  deep_block_score
count            220.000000        220.000000        220.000000
mean              40.142589         40.487914         46.583600
std               19.332464         19.868512         19.537384
min                0.000000          0.000000          0.000000
25%               26.256390         26.760787         33.863418
50%               39.080005         35.723383         44.980664
75%               51.066248         52.605384         58.113083
max              100.000000        100.000000        100.000000


In [16]:
merged_df['long_pass_attempts_per_tot_pass_attempts'] = merged_df['Long_Att'] / merged_df['Total_Att']
merged_df['PrgP_dist_per_tot_pass_dist'] = merged_df['Total_PrgDist'] / merged_df['Total_TotDist']

print(merged_df)


                 league  season             team  players_used  90s  \
0    ENG-Premier League    2324          Arsenal            25   27   
1    ENG-Premier League    2324      Aston Villa            28   27   
2    ENG-Premier League    2324      Bournemouth            27   26   
3    ENG-Premier League    2324        Brentford            28   27   
4    ENG-Premier League    2324         Brighton            27   27   
..                  ...     ...              ...           ...  ...   
215          ARG-League    2324        Sarmiento            32   27   
216          ARG-League    2324         Talleres            31   27   
217          ARG-League    2324            Tigre            27   27   
218          ARG-League    2324          Tucumán            26   27   
219          ARG-League    2324  Vélez Sarsfield            27   27   

     Standard_Gls  Standard_Sh  Standard_SoT  Standard_SoT%  Standard_Sh/90  \
0              65          453           148           32.7         

In [17]:
import pandas as pd
import numpy as np

def cap_extreme_values(data, columns):
    for col in columns:
        upper_limit = data[col].quantile(0.99)
        lower_limit = data[col].quantile(0.01)
        data[col] = np.clip(data[col], lower_limit, upper_limit)
    return data

def log_transform(data, columns):
    for col in columns:
        offset = 1e-5  # Avoid log(0) issue
        data[col + '_log'] = np.log(data[col] + offset)
    return data

def calculate_z_scores(data, columns):
    z_scores_df = data.copy()
    for col in columns:
        if 'long_pass_attempts_per_tot_pass_attempts' in col or 'PrgP_dist_per_tot_pass_dist' in col:
            z_scores_df[col + '_z'] = -(z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()  # Inversion
        else:
            z_scores_df[col + '_z'] = (z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()
    return z_scores_df

def compute_deep_circulation_score(data, columns, weights):
    z_score_columns = [col + '_z' for col in columns]
    data['deep_circulation_score'] = data[z_score_columns].dot(weights)
    min_score = data['deep_circulation_score'].min()
    max_score = data['deep_circulation_score'].max()
    data['deep_circulation_score'] = ((data['deep_circulation_score'] - min_score) / (max_score - min_score)) * 100
    return data

# Columns and weights configuration
deep_circulation_columns = [
    'long_pass_attempts_per_tot_pass_attempts', 
    'PrgP_dist_per_tot_pass_dist', 
    'Total_Cmp%'
]
weights_deep_circulation = np.array([0.4, 0.3, 0.3])  # Weighting considering importance

# Load data and apply transformations
merged_df = cap_extreme_values(merged_df, deep_circulation_columns)
merged_df = log_transform(merged_df, deep_circulation_columns)
scores_df_deep_circulation = calculate_z_scores(merged_df, deep_circulation_columns)
scores_df_deep_circulation = compute_deep_circulation_score(scores_df_deep_circulation, deep_circulation_columns, weights_deep_circulation)

# Merge into a new DataFrame
team_scores_df = team_scores_df.merge(scores_df_deep_circulation[['team', 'league', 'deep_circulation_score']],
                                      on=['team', 'league'], how='left')

# Display the DataFrame to check the results
print(team_scores_df.head())


          team              league  high_press_score  deep_block_score  \
0      Arsenal  ENG-Premier League         77.614792         51.250689   
1  Aston Villa  ENG-Premier League         34.251561          0.000000   
2  Bournemouth  ENG-Premier League         53.723397         25.881742   
3    Brentford  ENG-Premier League         32.431239         75.947849   
4     Brighton  ENG-Premier League         65.279580          9.840267   

   narrow_defense_score  deep_circulation_score  
0             47.643715               94.734974  
1             48.477344               82.572339  
2             16.406429               47.706227  
3             18.068076               52.797980  
4             17.682018               97.930934  


In [18]:
def view_top_players(df, score_column, num_players=10):
    """ Display the top players based on the given score column. """
    top_players = df.sort_values(by=score_column, ascending=False).head(num_players)
    return top_players

# Example usage to view the top 10 players for the 'Progressor' role
top_wcs = view_top_players(team_scores_df, 'deep_circulation_score')
print(top_wcs)


                team              league  high_press_score  deep_block_score  \
107        Paris S-G         FRA-Ligue 1         78.398995         39.372682   
148      Southampton    ENG-Championship         83.637795         30.448494   
12   Manchester City  ENG-Premier League         61.194807         20.604560   
4           Brighton  ENG-Premier League         65.279580          9.840267   
86        Leverkusen      GER-Bundesliga         60.474923         47.367208   
35       Real Madrid         ESP-La Liga         42.825005         77.906301   
77     Bayern Munich      GER-Bundesliga         72.413922         44.282925   
165       Fluminense          BRA-League         62.283890         50.243531   
0            Arsenal  ENG-Premier League         77.614792         51.250689   
24         Barcelona         ESP-La Liga         64.309017         31.084303   

     narrow_defense_score  deep_circulation_score  
107             28.329006              100.000000  
148            

In [19]:
# Display descriptive statistics for the three score columns
descriptive_statistics = team_scores_df[['narrow_defense_score', 'high_press_score', 'deep_block_score', 'deep_circulation_score']].describe()
print(descriptive_statistics)


       narrow_defense_score  high_press_score  deep_block_score  \
count            220.000000        220.000000        220.000000   
mean              40.142589         40.487914         46.583600   
std               19.332464         19.868512         19.537384   
min                0.000000          0.000000          0.000000   
25%               26.256390         26.760787         33.863418   
50%               39.080005         35.723383         44.980664   
75%               51.066248         52.605384         58.113083   
max              100.000000        100.000000        100.000000   

       deep_circulation_score  
count              220.000000  
mean                57.322289  
std                 21.175419  
min                  0.000000  
25%                 43.829482  
50%                 56.977947  
75%                 72.525266  
max                100.000000  


In [20]:
merged_df['att_3rd_touches_per_pen_area_touch'] = merged_df['Touches_Att 3rd'] / merged_df['Touches_Att Pen']

print(merged_df)


                 league  season             team  players_used  90s  \
0    ENG-Premier League    2324          Arsenal            25   27   
1    ENG-Premier League    2324      Aston Villa            28   27   
2    ENG-Premier League    2324      Bournemouth            27   26   
3    ENG-Premier League    2324        Brentford            28   27   
4    ENG-Premier League    2324         Brighton            27   27   
..                  ...     ...              ...           ...  ...   
215          ARG-League    2324        Sarmiento            32   27   
216          ARG-League    2324         Talleres            31   27   
217          ARG-League    2324            Tigre            27   27   
218          ARG-League    2324          Tucumán            26   27   
219          ARG-League    2324  Vélez Sarsfield            27   27   

     Standard_Gls  Standard_Sh  Standard_SoT  Standard_SoT%  Standard_Sh/90  \
0              65          453           148           32.7         

In [21]:
import pandas as pd
import numpy as np

def cap_extreme_values(data, columns):
    for col in columns:
        upper_limit = data[col].quantile(0.99)
        lower_limit = data[col].quantile(0.01)
        data[col] = np.clip(data[col], lower_limit, upper_limit)
    return data

def log_transform(data, columns):
    for col in columns:
        offset = 1e-5  # Avoid log(0) issue
        data[col + '_log'] = np.log(data[col] + offset)
    return data

def calculate_z_scores(data, columns):
    z_scores_df = data.copy()
    for col in columns:
        z_scores_df[col + '_z'] = (z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()
    return z_scores_df

def compute_high_retention_score(data, columns, weights):
    z_score_columns = [col + '_z' for col in columns]
    data['high_retention_score'] = data[z_score_columns].dot(weights)
    min_score = data['high_retention_score'].min()
    max_score = data['high_retention_score'].max()
    data['high_retention_score'] = ((data['high_retention_score'] - min_score) / (max_score - min_score)) * 100
    return data

# Columns and weights configuration
high_retention_columns = [
    'att_3rd_touches_per_pen_area_touch'  # Assuming this is the correctly named column
]
weights_high_retention = np.array([1.0])  # 100% weight

# Load data and apply transformations
merged_df = cap_extreme_values(merged_df, high_retention_columns)
merged_df = log_transform(merged_df, high_retention_columns)
scores_df_high_retention = calculate_z_scores(merged_df, high_retention_columns)
scores_df_high_retention = compute_high_retention_score(scores_df_high_retention, high_retention_columns, weights_high_retention)

# Merge into a new DataFrame
team_scores_df = team_scores_df.merge(scores_df_high_retention[['team', 'league', 'high_retention_score']],
                                      on=['team', 'league'], how='left')

# Display the DataFrame to check the results
print(team_scores_df.head())


          team              league  high_press_score  deep_block_score  \
0      Arsenal  ENG-Premier League         77.614792         51.250689   
1  Aston Villa  ENG-Premier League         34.251561          0.000000   
2  Bournemouth  ENG-Premier League         53.723397         25.881742   
3    Brentford  ENG-Premier League         32.431239         75.947849   
4     Brighton  ENG-Premier League         65.279580          9.840267   

   narrow_defense_score  deep_circulation_score  high_retention_score  
0             47.643715               94.734974             24.952434  
1             48.477344               82.572339              0.000000  
2             16.406429               47.706227              1.595617  
3             18.068076               52.797980              0.000000  
4             17.682018               97.930934             23.335098  


In [22]:
# Display descriptive statistics for the three score columns
descriptive_statistics = team_scores_df[['narrow_defense_score', 'high_press_score', 'deep_block_score', 'deep_circulation_score', 'high_retention_score']].describe()
print(descriptive_statistics)


       narrow_defense_score  high_press_score  deep_block_score  \
count            220.000000        220.000000        220.000000   
mean              40.142589         40.487914         46.583600   
std               19.332464         19.868512         19.537384   
min                0.000000          0.000000          0.000000   
25%               26.256390         26.760787         33.863418   
50%               39.080005         35.723383         44.980664   
75%               51.066248         52.605384         58.113083   
max              100.000000        100.000000        100.000000   

       deep_circulation_score  high_retention_score  
count              220.000000            220.000000  
mean                57.322289             43.523025  
std                 21.175419             24.088109  
min                  0.000000              0.000000  
25%                 43.829482             25.271541  
50%                 56.977947             41.074569  
75%               

In [23]:
def view_top_players(df, score_column, num_players=10):
    """ Display the top players based on the given score column. """
    top_players = df.sort_values(by=score_column, ascending=False).head(num_players)
    return top_players

# Example usage to view the top 10 players for the 'Progressor' role
top_wcs = view_top_players(team_scores_df, 'high_retention_score')
print(top_wcs)


                 team          league  high_press_score  deep_block_score  \
31         Las Palmas     ESP-La Liga         57.038022          9.672959   
209       Newell's OB      ARG-League         63.062055         76.522611   
200        CC Córdoba      ARG-League         20.070979         53.940656   
162          Cruzeiro      BRA-League         43.476938         61.727552   
165        Fluminense      BRA-League         62.283890         50.243531   
48           Kortrijk  BEL-Pro League         12.150248         17.592479   
159        Bragantino      BRA-League         87.473639         37.686548   
211       Racing Club      ARG-League         60.876761         39.728570   
156  Atlético Mineiro      BRA-League         51.876659         76.140812   
37            Sevilla     ESP-La Liga         40.894365         40.763008   

     narrow_defense_score  deep_circulation_score  high_retention_score  
31              62.454042               81.127362            100.000000  
209 

In [24]:
merged_df['switches_per_pass_att'] = merged_df['Pass Types_Sw'] / merged_df['Total_Att']
merged_df['crosses_per_att_3rd_touch'] = merged_df['Pass Types_Crs'] / merged_df['Touches_Att 3rd']

print(merged_df)


                 league  season             team  players_used  90s  \
0    ENG-Premier League    2324          Arsenal            25   27   
1    ENG-Premier League    2324      Aston Villa            28   27   
2    ENG-Premier League    2324      Bournemouth            27   26   
3    ENG-Premier League    2324        Brentford            28   27   
4    ENG-Premier League    2324         Brighton            27   27   
..                  ...     ...              ...           ...  ...   
215          ARG-League    2324        Sarmiento            32   27   
216          ARG-League    2324         Talleres            31   27   
217          ARG-League    2324            Tigre            27   27   
218          ARG-League    2324          Tucumán            26   27   
219          ARG-League    2324  Vélez Sarsfield            27   27   

     Standard_Gls  Standard_Sh  Standard_SoT  Standard_SoT%  Standard_Sh/90  \
0              65          453           148           32.7         

In [25]:
import pandas as pd
import numpy as np

def cap_extreme_values(data, columns):
    for col in columns:
        upper_limit = data[col].quantile(0.99)
        lower_limit = data[col].quantile(0.01)
        data[col] = np.clip(data[col], lower_limit, upper_limit)
    return data

def log_transform(data, columns):
    for col in columns:
        offset = 1e-5  # Avoid log(0) issue
        data[col + '_log'] = np.log(data[col] + offset)
    return data

def calculate_z_scores(data, columns):
    z_scores_df = data.copy()
    for col in columns:
        z_scores_df[col + '_z'] = (z_scores_df[col] - z_scores_df[col].mean()) / z_scores_df[col].std()
    return z_scores_df

def compute_high_retention_score(data, columns, weights):
    z_score_columns = [col + '_z' for col in columns]
    data['wide_attack_score'] = data[z_score_columns].dot(weights)
    min_score = data['wide_attack_score'].min()
    max_score = data['wide_attack_score'].max()
    data['wide_attack_score'] = ((data['wide_attack_score'] - min_score) / (max_score - min_score)) * 100
    return data

# Columns and weights configuration
wide_attack_columns = [
    'crosses_per_att_3rd_touch', 'switches_per_pass_att'  # Assuming this is the correctly named column
]
weights_wide_attack = np.array([.3, .7])  # 100% weight

# Load data and apply transformations
merged_df = cap_extreme_values(merged_df, wide_attack_columns)
merged_df = log_transform(merged_df, wide_attack_columns)
scores_df_wide_attack = calculate_z_scores(merged_df, wide_attack_columns)
scores_df_wide_attack = compute_high_retention_score(scores_df_wide_attack, wide_attack_columns, weights_wide_attack)

# Merge into a new DataFrame
team_scores_df = team_scores_df.merge(scores_df_wide_attack[['team', 'league', 'wide_attack_score']],
                                      on=['team', 'league'], how='left')

# Display the DataFrame to check the results
print(team_scores_df.head())


          team              league  high_press_score  deep_block_score  \
0      Arsenal  ENG-Premier League         77.614792         51.250689   
1  Aston Villa  ENG-Premier League         34.251561          0.000000   
2  Bournemouth  ENG-Premier League         53.723397         25.881742   
3    Brentford  ENG-Premier League         32.431239         75.947849   
4     Brighton  ENG-Premier League         65.279580          9.840267   

   narrow_defense_score  deep_circulation_score  high_retention_score  \
0             47.643715               94.734974             24.952434   
1             48.477344               82.572339              0.000000   
2             16.406429               47.706227              1.595617   
3             18.068076               52.797980              0.000000   
4             17.682018               97.930934             23.335098   

   wide_attack_score  
0           3.063576  
1          36.837456  
2          53.438740  
3          61.228419  
4

In [26]:
def view_top_players(df, score_column, num_players=10):
    """ Display the top players based on the given score column. """
    top_players = df.sort_values(by=score_column, ascending=False).head(num_players)
    return top_players

# Example usage to view the top 10 players for the 'Progressor' role
top_wcs = view_top_players(team_scores_df, 'wide_attack_score')
print(top_wcs)


              team              league  high_press_score  deep_block_score  \
182      Famalicão          POR-League         31.886630         52.856682   
28          Getafe         ESP-La Liga         42.293817         33.888733   
208          Lanús          ARG-League         55.103221         85.515126   
37         Sevilla         ESP-La Liga         40.894365         40.763008   
196       Belgrano          ARG-League         34.962998         52.792657   
18        West Ham  ENG-Premier League         19.712207         34.217045   
63   Hellas Verona         ITA-Serie A         34.894023         38.010818   
216       Talleres          ARG-League         30.143886         62.354158   
205     Godoy Cruz          ARG-League         40.364465         41.556726   
11      Luton Town  ENG-Premier League         30.651275         14.992956   

     narrow_defense_score  deep_circulation_score  high_retention_score  \
182             34.208261               42.361681             30.2

In [27]:
# Columns that contain scores
score_columns = ['high_press_score', 'deep_block_score', 'narrow_defense_score', 'deep_circulation_score', 'high_retention_score', 'wide_attack_score']

# Round each score column to two decimal places
for column in score_columns:
    if column in team_scores_df.columns:
        team_scores_df[column] = team_scores_df[column].round(2)

# Display the DataFrame to verify the changes
print(team_scores_df.head())


          team              league  high_press_score  deep_block_score  \
0      Arsenal  ENG-Premier League             77.61             51.25   
1  Aston Villa  ENG-Premier League             34.25              0.00   
2  Bournemouth  ENG-Premier League             53.72             25.88   
3    Brentford  ENG-Premier League             32.43             75.95   
4     Brighton  ENG-Premier League             65.28              9.84   

   narrow_defense_score  deep_circulation_score  high_retention_score  \
0                 47.64                   94.73                 24.95   
1                 48.48                   82.57                  0.00   
2                 16.41                   47.71                  1.60   
3                 18.07                   52.80                  0.00   
4                 17.68                   97.93                 23.34   

   wide_attack_score  
0               3.06  
1              36.84  
2              53.44  
3              61.23  
4

In [28]:
import pandas as pd
import subprocess

# Assuming RB_scores_df is defined elsewhere in your notebook
team_scores_df.to_csv('Team Playstyle Scores.csv', index=False)

def run_command(command):
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    if result.stderr:
        print("Error:", result.stderr)
    else:
        print("Output:", result.stdout)

# Properly quote the filename to handle spaces
run_command('git add "Team Playstyle Scores.csv"')
run_command('git commit -m "Add or update Team Playstyle Scores.csv"')

# Make sure the branch 'main' exists by checking your GitHub repo or by running 'git branch' locally.
run_command('git push origin main')  # Adjust branch name if necessary


Output: 
Output: [main 8993ecd] Add or update Team Playstyle Scores.csv
 1 file changed, 221 insertions(+)
 create mode 100644 Player Metrics/Team Stats/Team Playstyle Scores.csv

Error: To https://github.com/Jbuck1016/Player-Recommendation-System.git
   1914917..8993ecd  main -> main



In [29]:
import subprocess

def run_command(command):
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    if result.stderr:
        print("Error:", result.stderr)
    else:
        print("Output:", result.stdout)

# Add, commit, pull, and then push
run_command('git add "Creating Team Scores.ipynb"')
run_command('git commit -m "Update notebook with latest changes"')
run_command('git pull origin main')  # Pull before pushing to ensure the local repo is up-to-date
run_command('git push origin main')  # Adjust branch name if necessary


Error: warning: in the working copy of 'Player Metrics/Team Stats/Creating Team Scores.ipynb', LF will be replaced by CRLF the next time Git touches it

Output: [main 6da994e] Update notebook with latest changes
 1 file changed, 2302 insertions(+)
 create mode 100644 Player Metrics/Team Stats/Creating Team Scores.ipynb

Error: From https://github.com/Jbuck1016/Player-Recommendation-System
 * branch            main       -> FETCH_HEAD

Error: To https://github.com/Jbuck1016/Player-Recommendation-System.git
   8993ecd..6da994e  main -> main

